# TP1: Collection of Sentinel-2 and Landsat-8 Images

### Name: **WRITE YOUR NAME HERE**

The objective of this session is to create a video of your house seen from space using satellite images freely available online. For that we are going to learn how to access and manipulate optical images from the Sentinel-2 and Landsat-8 satellites, and radar images from Sentinel-1. We will cover the following topics:
* coordinate reference systems, WGS84 (longitude/latitude), and projections (Mercator, UTM, EPSG codes)
* search and download images available at a given position
* image metadata, GeoTIFFs, and GDAL (gdalinfo)
* spectral bands and file formats

#### Instructions
To solve this TP, answer the questions below. Then export the notebook with the answers using the menu option **File->Download as->Notebook (.ipynb)**. Then [submit the resulting file here](https://forms.gle/k6FSX2BrPix823dx6) by next week. You will receive an automatic acknowledgement of receipt.

The notes associated to this lesson are in [TP1.pdf](TP1.pdf).

There are **8 questions** in the notebook and corresponding text areas to fill-in the answers.

In [ ]:
## Setup code for the notebook
##
%matplotlib notebook
# Autoreload external python modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# These are the main includes used through the notebook
import numpy as np                   # numeric linear algebra
import matplotlib.pyplot as plt      # plotting
import rasterio       # read/write geotiffs
import tsd            # time series downloader for sentinel and landsat
import utils          # IO and coordinate system conversion tools
import vistools       # display tools

## A. Coordinate systems: longitude/latitude and UTM

* longitude/latitude and UTM cooridnates
* convert from long/lat to UTM and back
   - `utils.lonlat_to_utm(longitude, latitude, force_epsg=None)`: from lon/lat 
   - `utils.utm_to_lonlat(easting, northing, epsg)`: to lon/lat
* interactive map tool for navigation
   - `vistools.clickablemap(center_latlon, zoom=13)`: creates an interactive map and returns a map handle to interact with it.

In [ ]:
## First we will use a map to find the coordinates of a region of interest
##

print('Select a rectangular region of interest on the interactive map')

# these lines create an interactive map objet and display it
m = vistools.clickablemap(zoom=13)
display(m)

# we can move the map to any position by indicating its (latitude, longitude)
m.center = [48.841413, 2.328409]   # i.e. Paris, France

In [ ]:
## then we read the coordinates of the selection as a GeoJSON polygon
##

try:                # recover the polygon
    aoi = m.AOIs[-1]['geometry']

except IndexError:  # predefined AOI if nothing was selected
    # Pyramids:
    # aoi ={'coordinates': [[[31.12624503672123, 29.97234031281746], [31.12624503672123, 29.982786256861996], [31.14066459238529, 29.982786256861996],  [31.14066459238529, 29.97234031281746],
    #                       [31.12624503672123, 29.97234031281746]]], 'type': 'Polygon'}
    # SF financial district:
    aoi = {'type': 'Polygon', 'coordinates': [[[-122.410056, 37.77979], [-122.410056, 37.798578],
                        [-122.383276, 37.798578], [-122.383276, 37.77979]]]}

    # add a polygon and center map
    m.add_GeoJSON(aoi)         # this draws the polygon described by aoi
m.center =  aoi['coordinates'][0][0][::-1]

print("These are the coordinates of the selected polygon in GeoJSON format:   " + str(aoi))

**Warning: m.center uses a reversed ordering convention for longitude and latitude**

In [ ]:
print("lat, lon: {}".format(m.center))

**Exercise 1.**

UTM coordinates are interesting because they are locally Euclidean.

Implement the function `two_way_conversion_error` below, to convert (longitude, latitude) coordinates to UTM coordinates (easting, northing, and zone), then back to (longitude, latitude), and measure the resulting conversion error.

Use the `lonlat_to_utm` and `utm_to_lonlat` functions
from the `utils` module to convert to and from UTM.

```python
     utils.lonlat_to_utm(longitude, latitude, force_epsg=None)
     utils.utm_to_lonlat(easting, northing, epsg)
```

**NOTE:** be careful with the parameter conventions lon/lat vs lat/lon.

**Solution**

In [ ]:
def two_way_conversion_error(lon, lat):
    """
    Compute the error after converting to UTM and back to lon, lat.
    
    Args:
        lon (float): longitude
        lat (float): latitude
        
    Returns:
        the difference between input lon, lat and lon, lat obtained 
        after converting back and forth to UTM.
    """
    ###### Insert your solution code here ######


The code in the cell below extracts the longitude, latitude of the center of the map, and call your `two_way_conversion_error` function to measure the lon, lat --> UTM --> lon, lat conversion error.

In [ ]:
# call your function
lat, lon = m.center
error = two_way_conversion_error(lon,lat)
np.testing.assert_allclose(error, np.zeros(2), atol=1e-9)
print(error)

**Exercise 2.**
Trace a 200km west-to-east line (with constant northing) in UTM coordinates with 10 samples.

Start at: `easting=400000; northing=5400000; epsg=32631`

Convert it to lon/lat and plot the resulting coordinates in a cartesian lon/lat plot.

Plot the same line on the map by using

    line = {'type': 'LineString', 'coordinates': [[-122.6, 37.], [..., ...], [-122., 37.]]}
    m.add_GeoJSON(line)

What do you observe? Why?

Bear in mind these conclusions about the projections.

**Solution**

In [ ]:
###### Insert your solution code here ######


# trace and convert the line to lon/lat

coordinates = list(zip(lons, lats))

# write the geoJSON and show in the map
line = {'type': 'LineString'}
line['coordinates'] = coordinates
m.add_GeoJSON(line)
m.center= coordinates[0][::-1]

# plot lon/lat
plt.figure()
plt.plot(lons, lats)
plt.title('lon/lat plot of an east-to-west line in UTM coordinates')

display(m)

**Question:** What do you observe in the previous experiment? Why?

In [ ]:
###### Write your answer here ######

## B. Query an image database for an AOI (area of interest)

We want to find all the available images over an area of interest (AOI). The Python package `tsd` (Time Series Downloader) allows to search for Sentinel-2 and Landsat-8 images on free online catalogs (such as scihub, devseed, planet...). 

To query the online catalog for Landsat-8 and Sentinel-2 images:
- `tsd.get_sentinel2.search`: search the Sentinel-2 online catalog
- `tsd.get_landsat.search`: search the Landsat-8 online catalog

Each catalog entry is returned by `tsd` as a Python dictionary containing the image metadata. To display nicely the list of dictionaries, one may turn it into a `pandas.DataFrame`.

In [ ]:
# SanFrancisco financial district
aoi = {'coordinates': [[[-122.410056, 37.77979], [-122.410056, 37.798578],
                        [-122.383276, 37.798578], [-122.383276, 37.77979],
                        [-122.410056, 37.77979]]], 'type': 'Polygon'}

# Run the query
image_catalog = tsd.get_sentinel2.search(aoi)
#image_catalog = tsd.get_landsat.search(aoi)

if len(image_catalog) == 0:
    print("Maybe your AOI selection is too large")

# There's plenty of data associated to each entry of the catalog.
# Take some time to see all the available keys of each image entry dict.
# Note that there is a cloud cover percentage estimation,
# and a thumbnail url
import pandas as pd
pd.DataFrame([dict(i) for i in image_catalog])

The catalog contains the URL of each full image, thumbnail, and other information useful to filter the results before downloading the images.

To display an image in the notebook, use the `vistools.display_image` function:
- `vistools.display_image`: display images contained in np.arrays, files, or urls

In [ ]:
## Let's select the last entry from the catalog
entry = image_catalog[-1]

# and display its thumbnail
vistools.display_image(entry['thumbnail'])   #  for displaying an image in the notebook
print('At any moment, about 60 percent of the earth is covered by clouds.')

**Exercise 3.**
Implement the function `query_clear_sky` according to the interface specified below. The function must query for Sentinel-2 images on the provided aoi and return only those with a cloud cover below the provided percentage.


```python
def query_clear_sky(aoi, satellite='Sentinel-2', max_cloud_cover=30 ):
    '''
    queries the devseed database for the aoi
    returns a filtered catalog with images with cloud
    cover of at most max_cloud_cover
    my_satellite:   'Landsat-8'  or 'Sentinel-2'
    '''
    # run a search
    if satellite == 'Sentinel-2':
        res = tsd.get_sentinel2.search(aoi)
    elif satellite == 'Landsat-8':
        res = tsd.get_landsat.search(aoi)


    ###     <<< CODE HERE >>>


    return out   # returns the filtered catalog
```

**Hint:** access  fields from the registers using attributes i.e. `image_catalog[0].date`

**Solution**

In [ ]:
# Ex 3.
def query_clear_sky(aoi, satellite='Sentinel-2', max_cloud_cover=30 ):
    '''
    queries the devseed database for the aoi
    returns a filtered catalog with images with cloud
    cover of at most max_cloud_cover
    my_satellite:   'Landsat-8'  or 'Sentinel-2'
    '''
    # run a search
    if satellite == 'Sentinel-2':
        res = tsd.get_sentinel2.search(aoi)
    elif satellite == 'Landsat-8':
        res = tsd.get_landsat.search(aoi)


    ###### Insert your solution code here ######



    return res

This queries for images containing the AOI with cloud cover of at most of 30%, and displays the THUMBNAIL of all the images.

In [ ]:
#### Test your function by calling it with
aoi = {'coordinates': [[[-122.410056, 37.77979], [-122.410056, 37.798578],
                        [-122.383276, 37.798578], [-122.383276, 37.77979],
                        [-122.410056, 37.77979]]], 'type': 'Polygon'}

# Let's display all the images with cloud cover < 30
res = query_clear_sky(aoi, satellite='Sentinel-2', max_cloud_cover=10)

## display the images in a gallery
g, l = [], []
for p in res:
    l.append((p.date, p.cloud_cover))
    g.append(p.thumbnail)


vistools.display_gallery(g, l)

## C. Downloading a full resolution image

Since the images are usually quite large (40k x 40k pixels, about 1GB) instead of downloading the entire images **tsd** (through GDAL) allows to download just a crop containing the desired AOI.

The `get_crop_from_aoi` function is a simplified interface to GDAL for cropping an AOI. Internally, it
* Extracts the url of the image from the catalog entry for the selected band
* Downloads a crop corresponding to the selected aoi
* Writes a GeoTIFF

The underlying function for cropping an image on the web is `tsd.utils.crop_with_gdal_translate(output, inpath, ulx, uly, lrx, lry, utm_zone=None, lat_band=None)`.

In this lower level function the aoi is specified using UTM bounding box coordinates `(ulx, uly, lrx, lry, utm_zone, lat_band)` **but today we will only use it indirectly**.

In [ ]:
def get_crop_from_aoi(output_path, aoi, catalog_entry, band):
    """
    Crop and download an AOI from a georeferenced image.

    Args:
        output_path (string): path to the output GeoTIFF file
        aoi (geojson.Polygon): area of interest defined by a polygon in longitude, latitude coordinates
        catalog_entry (tsd.s2_metadata_parser.Sentinel2Image): metadata object
        band (str): desired band, e.g. 'B04' for Sentinel-2 or 'B8' for Landsat-8
    """
    metadata = catalog_entry
    if not metadata.urls['gcloud']:
        metadata.build_gs_links()
    inpath = metadata.urls['gcloud'][band]
    utm_zone = int(metadata.utm_zone) if 'utm_zone' in metadata else None
    ulx, uly, lrx, lry, utm_zone, lat_band = tsd.utils.utm_bbx(aoi, utm_zone=utm_zone,  r=60)
    tsd.utils.crop_with_gdal_translate(output_path, inpath, ulx, uly, lrx, lry, utm_zone, lat_band)

In [ ]:
# Let's start downloading the green channel
my_satellite, band = 'Sentinel-2', 'B03' # 'Landsat-8', 'B8' #
fname = 'ex4_{}.tif'.format(band)
results = query_clear_sky(aoi, my_satellite, max_cloud_cover=30)

# This is where the downloading is done
get_crop_from_aoi(fname, aoi, results[-1], band)

# To read the image and metadata useutils.readGTIFF andutils.readGTIFFmeta
im   = utils.readGTIFF(fname)
meta = utils.readGTIFFmeta(fname)

print('We can see some metadata...')
display(meta)
print('...and the image. However it has a high bit depth so it cannot be shown as it is')
vistools.display_image(im)
print('we can divide by 32 and see what happens')
vistools.display_image(im/32)
print('the result seems ok, but a bit dark')

## D. Read, write, and display images
* GeoTIFF and 8 bits formats (PNG and JPEG)
* read/write *GeoTIFF*s and its metadata:
  -  `utils.readGTIFF(fname)`:   Reads the image file `fname` into a numpy array,
    returns the numpy array with dimensios (height, width, channels)
  - `utils.readGTIFFmeta(fname)`: Reads the GeoTIFF metadata and returns a tuple
    containing the metadata and bounding box: (meta, bounds)
    if the file format doesn't support metadata the returned metadata is invalid.
  -  `utils.writeGTIFF(im, fname, copy_metadata_from=None)`:  Writes a numpy array to a *GeoTIFF*, *PNG*, or *JPEG* file depending on `fname` extension.
    For GeoTIFF files the metadata can be copied from another file (optional parameter).
    Note that if  `im`  and  `copy_metadata_from` have different size,
    the copied geolocation properties are not adapted.
* display a numpy array with a color palette:
  - `vistools.display_imshow(out,[0,1], cmap="jet")`: display the `[0,1]` range using the `jet` palette
* assemble color images from bands
* compute a multispectral vegetation index: NDVI

**Exercise 4.**
The TIFF images can have a high dynamic range. To display them on
the screen we must map them to 8bits.
Write a program that affinely maps a high dynamic range image to
8bits (0 to 255). Determine the range of the input data and set
the saturation values at X and (100-X) percentile of the image.
Implement the function `simple_equalization_8bit` according to the interface specified below.


```python
def simple_equalization_8bit(im, percentiles=5):
    ''' im is a numpy array
        returns a numpy array
    '''

    ###     <<< CODE HERE >>>

    return im  # return equalized image
```

**Solution**

In [ ]:
def simple_equalization_8bit(im, percentiles=5):
    ''' im is a numpy array
        returns a numpy array
    '''
    import numpy as np

    return im   # return equalized image

This cell tests the above function.

In [ ]:
#### Test the function by calling it with
basename='ex4_'
im = utils.readGTIFF(basename+'B03.tif')
vistools.display_image(
    simple_equalization_8bit(im, percentiles=2)
)

In [ ]:
#  Let's write a function that downloads the Sentinel-2 RGB bands
#  and creates an 8bit color image. For that we will apply the function
#  simple_equalization_8bit to each channel.

def get_sentinel2_color_8bit(basefilename, aoi, catalog_entry, percentiles=5):
    ''' basefilename to store the bands:  basename+BAND+'.tif'
        returns a numpy array of the RGB image (height, width, channels)
    '''
    bands = ['B02', 'B03', 'B04']    # SENTINEL2 R,G,B BANDS

    # this command downloads all the bands
    for b in bands:
        get_crop_from_aoi('{}_{}.tif'.format(basefilename, b), aoi, catalog_entry, b)

    # read, equalize, and stack all the channels
    out = []
    for b in bands:
        im = utils.readGTIFF('{}_{}.tif'.format(basefilename, b))
        im = simple_equalization_8bit(im, percentiles)
        out.append(im)

    # The transposition is necessary because the indexing
    # convention for color images is (height, width, channels)
    im = np.squeeze(out,axis=(3)).transpose(1,2,0)
    return im


#### Test the function by running
# pick the satellite then query the database
my_satellite = 'Sentinel-2' # 'Landsat-8' #
basename = 'rgb'
res = query_clear_sky(aoi, my_satellite, max_cloud_cover=10)

# generate the RGB image
RGBout = get_sentinel2_color_8bit(basename, aoi, res[-1])

# Writes RGBout in 'rgb_RGB.tif' copying geolocation metadata from 'rgb_B03.tif',
# which has been written by    get_sentinel2_color_8bit
utils.writeGTIFF(RGBout[:,:,0:3], basename+'_RGB.tif', basename+'_B03.tif')

# display RGBout
vistools.display_imshow(RGBout[:,:,0:3])

**Exercise 5.**
  Multi-spectral images allow to measure things that are otherwise non visible.
  By using the NIR and RED bands, the normalized difference vegetation index (NDVI)
  (https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index) can assess
  whether the observed target contains live green vegetation or not.
  More on NDVI: https://phenology.cr.usgs.gov/ndvi_foundation.php
  Other indices can be found here:
  * http://www.sentinel-hub.com/apps/wms/wms-parameters/EOproducts
  * https://www.indexdatabase.de/db/ias.php

Let us compute and display the NDVI index on the AOI to find the parks and forests in a the area.
Implement the function `get_sentinel2_NDVI` according to the interface specified below.


```python
def get_sentinel2_NDVI(basefilename, aoi, catalog_entry):
    '''
    basefilename: to store the bands:  basename+BAND+'.tif'
    aoi: is the area of interest
    catalog_entry: is the register of a query
    returns a numpy array
    '''

    ###     <<< CODE HERE >>>

    return NDVI  # NDVI image (numpy array)
```

**Solution**

In [ ]:
# Ex 5.

def get_sentinel2_NDVI(basefilename, aoi, catalog_entry):
    ''' basefilename to store the bands:  basename+BAND+'.tif'
        returns a numpy array
    '''


    return NDVI   # NDVI image (numpy array)

The following lines allow to test the function.
This command:

      display_imshow(NDVI, [0,1], cmap="jet")
is used to show the array mapping the range [0,1] using the 'jet' colormap

In [ ]:
# pick the satellite then query the database
my_satellite = 'Sentinel-2' # 'Landsat-8' #
basename = 'ex5'
res = query_clear_sky(aoi, my_satellite, max_cloud_cover=10)

out = get_sentinel2_NDVI(basename, aoi, res[-1])

utils.writeGTIFF(out, basename+'_NDVI.tif', basename+'B08.tif')

vistools.display_imshow(out, [0,1], cmap="jet")

## E. Time series
* Downloading crops of all the images for an AOI could be done with the tools presented so far:

  `tsd.get_sentinel2.search`, `tsd.get_landsat.search` and `get_crop_from_aoi`.

  But, we provide the following two functions to simplify the task of searching and downloading all the images on the `aoi`:

   - `tsd.get_sentinel2.get_time_series(aoi, out_dir='tmp', bands=['B04'])`: for Sentinel-2 images
   - `tsd.get_landsat8.get_time_series`: The analogous function for Landsat-8.

* Make an animated gif

In [ ]:
# FIRST REMOVE THE OUTPUT FILES FROM PREVIOUS RUNS
!rm -rf tmp/*.tif.png* tmp/*.tif

# Download all the image for a satellite over an aoi
my_satellite = 'Sentinel-2' # 'Landsat-8' #
if my_satellite == 'Sentinel-2':
    tsd.get_sentinel2.get_time_series(aoi, out_dir='tmp', bands=['B03'], cloud_masks=False, api='scihub')
elif my_satellite == 'Landsat-8':
    tsd.get_landsat.get_time_series(aoi, out_dir='tmp', bands=['B8'], cloud_masks=True)

import glob

# list the donwloaded files
#print (glob.glob('tmp/*.tif') )


# select the green band 'B03' (sentinel), or panchromatic '8' (landsat)
# then read tiff and convert them to png (geolocation information is lost)
for i in  glob.glob('tmp/*_B8.tif') + (glob.glob('tmp/*B03.tif')):
    im = utils.readGTIFF( i )
    im = simple_equalization_8bit(im,  percentiles=2)
    utils.writeGTIFF(im,'%s.png'%i)
    vistools.overprintText('%s.png'%i,'%s.png'%i,i[4:-4],textRGBA=[0,255,0,255])


# create an animated gif
!convert -delay 20 -loop 0 tmp/*.tif.png tmp/timeseries.gif

vistools.display_image('tmp/timeseries.gif')

### SAR time series

* obverve the modulus of an SLC radar image in a time series
* SLC images appear stretched in the azimuth direction (flight direction)

In [ ]:
## The files 'data/SF/SanFranciscoRadar{01-48}.tif' form a time series of sentinel-1 images
## the images are georeferenced and are already real with 8-bits values

import glob
# generate PNG
imgs=glob.glob('data/SF/SanFranciscoRadar??.tif')
for i in imgs:
    utils.writeGTIFF(utils.readGTIFF(i),i[:-3]+'png')
# display the result
imgs=sorted(glob.glob('data/SF/SanFranciscoRadar??.png'))
vistools.display_gallery(imgs)

**Exercise 6.**
  Simple denoising. Write a program to remove some of the noise by using the fact that the images are aligned.
  The simplest denoising consists in taking the average, however the geometric mean
  is more appropriate for the type of noise present in SAR images.

Implement the function `geometric_mean_images` according to the interface specified below.


```python
def geometric_mean_images(imgs_names_list):
    '''
    receives a list of image filenames
    returns a numpy array with the pixel-wise geometric mean
    '''

    ###     <<< CODE HERE >>>

    return gm
```

**Solution**

In [ ]:
def geometric_mean_images(imgs_names_list):
    '''
    receives a list of image filenames
    returns a numpy array with the pixel-wise geometric mean
    '''

    return gm

In [ ]:
#### Test the function by running
import glob
imgs=sorted(glob.glob('data/SF/SanFranciscoRadar??.tif'))

vistools.display_image(
    simple_equalization_8bit(
        geometric_mean_images(imgs)
    ))

## F. Reprojection

* determine the geographic coordinates (longitude, latitude, and altitude wrt the WGS84 reference
    ellipsoid) corresponding to an image coordinate (in pixels)
  - `utils.gdal_get_longlat_of_pixel(fname, x, y )`: returns the long/lat (at 0 altitude) for the points at pixel coordinates (x, y) of the image fname.
* resample (or reproject) a GeoTIFF in longitude, latitude coordinates
  - `utils.gdal_resample_image_to_longlat(fname, outfname)`: reproject GeoTIFF image in the long/lat image CRS

**Exercise 7.**
Determine the geographic coordinates (longitude, latitude) of the
 four corners of the SAR image and draw a polygon on a map.
 Write the function `get_image_longlat_polygon` following the prototype below and use provided test code.


```python
def get_image_longlat_polygon(filename):
    '''
    receives a GeoTIFF filename. Determines the image size
    and computes the long/lat coordinates of the four image corners.
    returns a GeoJSON polygon with the four corners
    '''

    ###     <<< CODE HERE >>>

    return polygon
```

**Hint:** use  `utils.gdal_get_longlat_of_pixel`

**Hint2:** make sure to use `utils.readGTIFFmeta`  in order to determine the
         size of the image. It is much faster than reading the whole image

**Solution**

In [ ]:
### Ex 7.

def get_image_longlat_polygon(filename):
    '''
    receives a GeoTIFF filename. Determines the image size
    and computes the long/lat coordinates of the four image corners.
    returns a GeoJSON polygon with the four corners
    '''

    coords = [ [x[0], x[1]] for x in coords ]

    polygon = {'type': 'Polygon', 'coordinates': [coords]}
    return polygon

In [ ]:
#### Test the function by running
fname = 'data/SF/SanFranciscoRadar01.tif'

footprint = get_image_longlat_polygon(fname)

# this creates a map and display the footprint polygon
m = vistools.clickablemap()

m.add_GeoJSON(footprint)
m.center = footprint[ 'coordinates'][0][0][::-1]
display(m)

In [ ]:
##  Now we can reproject the image in geographic coordinates (longitude, latitude)
##  using utils.gdal_resample_image_to_longlat, then compute the footprint of the
##  reprojected image and display the result on a map.

img = 'data/SF/SanFranciscoRadar01.tif'

# Reproject the image in longlat (one GeoTIFF and one PNG)
utils.gdal_resample_image_to_longlat(img, 'tmp/sarimage_longlat.tif')

# Extract the new image footprint from the GeoTIFF
footprint2 = get_image_longlat_polygon('tmp/sarimage_longlat.tif')
print (footprint2)

# Convert TIFF to 8-bits and write a PNG
data = utils.readGTIFF('tmp/sarimage_longlat.tif')
utils.writeGTIFF( simple_equalization_8bit( data ), 'tmp/sarimage_longlat.png' )

# Display the reprojected PNG overlaid on a map at the coordinates of the footprint
mo = vistools.overlaymap(footprint2, 'tmp/sarimage_longlat.png' , zoom = 13)
display(mo)

**Exercise 8.**
Read the GeoTIFF `data/fin.tif`, and overlay it on the map at its correct position and orientation

**Solution**

In [ ]:
img = 'data/fin.tif'


###### Insert your solution code here ######



In [ ]:
plt.close('all')

---------------------------
[//]: # (© 2018 Gabriele Facciolo)
[//]: # (<div style="text-align:center; font-size:75%;"> Copyright © 2018 Gabriele Facciolo. All rights reserved.</div> )